# Jupyter Playbooks for Suricata

Author: Markus Kont

## Introduction

Back in 2022, I did a Suricon presentation titled [Jupyter Playbooks for Suricata](https://youtu.be/hevTFubjlDQ). The goal of that presentation was to introduce JupyterLab to security practitioners who work with Suricata. Many people are familiar with exploring Suricata EVE output using established technology stacks such as Elastic stack, Splunk, etc. Yet might be unfamiliar with tools from data science world. Amazingly, a lot of people are still totally unfamiliar with EVE NSM data. It's 2023 and Suricata is still considered by many to be only a rule based IDS, even going so far that Suricata is often deployed in tandem with NSM logging tools as people believe Suricata is unable to fill the role.

To amend the situation, the presentation was focused around use-cases around exploring Suricata EVE JSON logs. It attempted to bridge a gap between threat hunting and data analysis, communities that have large overlap in what they do, yet remain quite far separated. It also attempted to highlight useful insights that can be extracted from EVE NSM data.

Original presentation resources can be found [here](https://github.com/StamusNetworks/suricata-analytics/blob/main/jupyter/Notebooks/Suricon2022Talk.ipynb). Since the presentation was about Jupyter notebooks, and Jupyter notebooks are by nature interactive, then it made no sense to me to format it as simple slideshow. In fact, notebook proved to be quite flexible presentation environment, and it turned the talk into one big tech demo. Perfect for a technical community conference. Input data was drawn from public sources, meaning anyone could run the notebook and repeat everything shown on stage.

That notebook was meant to be used as a resource. One that anyone could access and use as a reference for analyzing Suricata EVE logs. However, as anyone familiar with doing presentations knows, the challenge is not creating content. It's fitting what you have into the time window. A 45 minute tech talk really does not leave enough time to properly explain important concepts, especially if the audience is unfamiliar with talking points. Many ides were cut, others were explained quickly to move forward to interesting sections. Each pseudo-slide also needed to be readable, meaning no extensive *walls of text* as they would simply not fit the screen. The fact is, what works in a technical writing does not work for presenting. And vice versa.

Solution - write a extended notebook. More topics, more examples, and more context around each code cell. We will also section off the notebook into smaller blog posts for those not interested in interacting with notebook itself. Eventually it will formulate a series describing most important topics covered by the notebook. Why *most*? Because notebook can keep evolving over time, adding sections, improving examples, and maybe even reworking entire sections.

## About Jupyter notebooks

[Project Jupyter](https://jupyter.org/), born out of IPython Notebooks project, promotes open standards for interactive computing. Jupyter notebook is built on IPython kernel for running computations on interactive code inputted by the user, and outputs the results of those computations. In addition to code and output, user can also write markdown text to document each step. Hence the *notebook* moniker. Quick code to feedback loop makes Jupyter notebooks perfect for experimentation and documentation. It has become the de'facto tool for data scientists for this very reason. *Kernels* for many languages are supported. For example, [R](https://www.r-project.org/), [NodeJS](https://nodejs.org/en/), [Julia](https://julialang.org/), [Go](https://go.dev/) and [many more](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels).

### Basic concepts

A notebook is organized into *cells*. Those cells are generally executed from top to bottom, but can also be evaluated individually by the user. A cell can contain either `code` or `markdown`, but not both. 

#### Code Cells

A code cell is similar to simple IDE, as it allows programmer to write code while also providing syntax highlighting and autocompletion. A code cell, unlike a typical script file, can be executed individually. In other words, cells *should* be evaluated sequentially but user is free to jump to earlier cell while retaining variables in memory that were created by latter cells. Whenever a code cell is executed, it displays return value of last line of code withing that cell. Simple variables can even be called with no additional printing or formatting code, as notebook automatically catches the return value. It even has built-in display formatting for data frames. More on that later.

#### Markdown Cells

Markdown cells are simply for formatting text. Unlike code cells, they do not display the source and output separately. Active markdown cell displays raw markdown source for editing while executing those cells formats them for reading. User can easily toggle between editing and reading, unlike many other editors that only display the source and require generating a fully formatted document for reading.

#### Kernel

A *kernel* lies at the core of code evaluation loop. Jupyter itself is written in Python and [IPython](https://ipython.org/) is the default kernel it was built around. However, users can install any number of kernels for different languages. Those kernels can vary in maturity and quality. Don't be surprised when a kernel for your favorite language is missing basic features such as syntax highlighting or code suggestions. A custom kernel simply might not support those features yet. Some kernels might even be missing basic language features.

Kernel is chosen when first creating a notebook and can not be changed later. Using multiple kernels in a single notebook is not currently possible. Nor is it possible to change a kernel of existing notebook.

#### JupyterLab

*IPython notebook files* that use `.ipynb` file extension are generally considered to be the *Jupyter Notebooks*. However, the original web interface created for interacting with them is nowadays also often referred to as *jupyter notebook*. Reason is likely to distinguish that interface from [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html), an advanced interface with IDE-like features. Such as tabs, splits, variable exploration, extensions, and more. Old interface simply focused on interacting with a single notebook. A notebook *file* can be edited with both interfaces. There are no compatibility differences. JupyterLab simply provides a more extensive (but also more complex) interface.

Lab instance can be launched with `jupyter lab` command while legacy interface can be executed with `jupyter notebook`. Nowadays most projects default to using lab over simple notebook editor, but many people might still prefer the simplicity of the old interface. Might be worth considering when only getting started.

### Setup

Jupyter core, lab IDE, and most extensions can simply be installed using normal python tooling. Keep in mind that `jupyter` and `jupyterlab` are separate packages with latter extending the former. They can be installed with any python package manager, such as pip, conda, mamba, etc.

```
pip install jupyter jupyterlab
```

#### Local user install

Global system install with `pip` is not recommended. I would suggest installing as regular user if not using a python virtual environment.

```
python3 -m pip install --user jupyter jupyterlab
```

Please keep in mind that this method shall place `jupyter` command into `~/.local/bin`. Don't be surprised when unable to find the jupyter command. Use full path or add this folder to PATH for convenience.

```
export PATH="$HOME/.local/bin:$PATH"
```

```
># which jupyter
/home/user/.local/bin/jupyter
```

Once installed, simply run the jupyter command. Then mind the output.

```
(general) ➜  suricata-analytics-1 git:(next-suricon-2022-10-28) ✗ jupyter lab
[I 2022-10-30 06:10:48.141 ServerApp] jupyterlab | extension was successfully linked.
[I 2022-10-30 06:10:48.150 ServerApp] nbclassic | extension was successfully linked.
[I 2022-10-30 06:10:48.170 LabApp] JupyterLab extension loaded from /home/markus/venvs/general/lib/python3.10/site-packages/jupyterlab
[I 2022-10-30 06:10:48.170 LabApp] JupyterLab application directory is /home/markus/venvs/general/share/jupyter/lab
[I 2022-10-30 06:10:48.173 ServerApp] jupyterlab | extension was successfully loaded.
[I 2022-10-30 06:10:48.177 ServerApp] nbclassic | extension was successfully loaded.
[I 2022-10-30 06:10:48.177 ServerApp] The port 8888 is already in use, trying another port.
[I 2022-10-30 06:10:48.178 ServerApp] Serving notebooks from local directory: /home/markus/Projects/SN/suricata-analytics-1
[I 2022-10-30 06:10:48.178 ServerApp] Jupyter Server 1.21.0 is running at:
[I 2022-10-30 06:10:48.178 ServerApp] http://localhost:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
[I 2022-10-30 06:10:48.178 ServerApp]  or http://127.0.0.1:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
[I 2022-10-30 06:10:48.178 ServerApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 2022-10-30 06:10:48.216 ServerApp]

    To access the server, open this file in a browser:
        file:///home/markus/.local/share/jupyter/runtime/jpserver-395207-open.html
    Or copy and paste one of these URLs:
        http://localhost:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
     or http://127.0.0.1:8889/lab?token=b675c4daec9a6c2beb11b0a6cd38a314509ae62b1989b2e2
Opening in existing browser session.
```

Jupyter will by default launch the system default web browser. However, do pay special mind for the access information presented in this log output. Jupyter uses a randomly generated token to authenticate the session. This can of course be reconfigured in the configuration file to be a static token or even a password, but left unchanged requires following the connection link in order to properly authenticate. 

Do note also that example displays a port `8889` which is not the Jupyter default `8888`. Jupyter will detect if port is already in use, presumably by another instance of JupyterLab. When that happens, it simply increments the port number by one. Why would you want multiple instances of JupyterLab? Because you might want to use a different instances per project.

### Configuration

Jupyter supports extensive configuration. By default, the configuration file is missing and default values are implicitly used. To customize the setup, user will need to generate it.

```
jupyter notebook --generate-config
```

Default configuration file will be placed under `~/.jupyter/jupyter_notebook_config.py`. Notice that it's actually a python code file rather than plaintext configuration. This can enable very nice extensions to configuration. For example, a user might want to read configuration from environment variables instead which is useful for creating docker containers.

Customizing the configuration is not really needed for simple data exploration, however. Just something to keep in mind for advanced deployments.

### Code example

Jupyter is a interactive coding environment, so let's explore basic usage. Since the theme of this notebook is exploring EVE data, a good first exercise would be to download a PCAP file and parse it with Suricata. Results can then be analyzed in upcoming sections.

A great resource for PCAP files is [Malware Traffic Analysis](https://malware-traffic-analysis.net/), a site maintained by Suricata community member that hosts PCAP files for various malware infection scenarios. We start with a simple one that contains web scanning and [Log4j](https://www.cisa.gov/uscert/apache-log4j-vulnerability-guidance) CVE exploitation attempts. As any server administrator knows, website scanning is not really interesting traffic. It is inevitable when hosting any public services, scanning and exploitation attempts are fully automated by the malicious *spiders*. Think of it as malicious version of Google indexing your pages. All that can really be done against it is reducing the attack surface, keeping to best practices, and ensuring exposed services are fully up to date with latest patches. A exponentially more difficult task than those uninitiated would assume.

Nevertheless, that noise means abundance of data, making the PCAP perfect for displaying what Jupyter can do. We need to understand the general nature of the raw data, then separate relevant events from the noise.

In [1]:
URL = "https://malware-traffic-analysis.net/2022/01/03/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap.zip"

Archive can be downloaded with HTTP GET request using python `requests` library. 

In [2]:
import requests
response = requests.get(URL, stream=True)

Now we are able to conditionally check if download succeeded or not. If the response was HTTP 200 OK, then we stream the response payload into a local file handle. Along the way we also calculate some useful information, such as download size in kilobytes. If the response does not indicate a success, we simply report the failure. Note that failure reporting is done here mostly for demonstration. Most notebooks leave errors not handled, as subsequent code cells might depend on success on ones preceding it. Code evaluation is stopped when a cell *throws an exception*, prompting the user to figure out the issue. 

In [3]:
OUTPUT = "/tmp/malware-pcap.zip"

In [4]:
if response.status_code == 200:
    print("Download good, writing %d KBytes to %s" % 
          (int(response.headers.get("Content-length")) / 1024,
           OUTPUT))
    with open(OUTPUT, 'wb') as f:
        f.write(response.raw.read())
    print("Done")
else:
    print("Demo effect has kicked in")

Download good, writing 1254 KBytes to /tmp/malware-pcap.zip
Done


Once downloaded, we can simply use native python libraries to unzip the file. Scripting this rather than unzipping manually has several perks. For instance, threat research file archives that could contain actual malware samples are conventionally password protected, in order to protect unsuspecting users from compromising themselves. Standard password for these archives is `infected`. We can simply script this common password into unpacking call to save time.

Most difficult aspect about working with notebooks is dealing with data input and intermediate dumps. Reading a prepared CSV or JSON file is easy, but bundling it with notebook is not. Often I come back to a notebook that was made months ago, only to discover that it depends on data files that are no longer available. And nobody can remember any more how they were made. Or the notebook might point to hardcoded paths that only exist on analysts computer. It makes sense, since analyst wants to focus on the problem and not waste time dealing with how the data gets into the notebook. But that can make many notebooks unusable later.

It's a tough challenge and a balancing act. It is okay to make rough notebooks that are discarded after use. Data exploration is a fluid discipline, so properly documenting initial shots into the dark is not often worth the effort. This notebook, however, is not meant for that. Packaging how the data gets into the notebook can be just as important as the analysis.

In [5]:
from zipfile import ZipFile

In [6]:
with ZipFile(OUTPUT, "r") as zip:
    zip.extractall(path="/tmp", pwd="infected".encode("utf-8"))

Finally, we can verify that unpacked PCAP is in the filesystem by using *glob* search for finding all files with `.pcap` suffix. This allows us to build a variable that lists input files we can work with.

In [7]:
import glob

In [8]:
FILES = glob.glob("/tmp/*.pcap")
FILES

['/tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap']

Note that we can simply display the value of `FILES` variable by calling it. No need for any `print` or string formatting statements, although those could make the output look nicer. Notebook calls builtin `display()` method for best effort output visualization. Method call is implicit when in global scope, meaning user does not need to import the method nor call it. But it would have to be called when displaying data from a function. Keep that in mind.

Currently the output is a simple python list, so notebook displays it as such. No fancy formatting. However, we will soon see how data tables are automatically made nice looking. It could also be copy-pasted into code block or even into other scripts or programs. Jupyter can and often is used for generating code that would otherwise be too tedious to write by hand.

### Invoking a shell command

Python code displayed in prior section is not complicated. But not even the most experienced programmers know every API by memory. Quite often we need to resort to scouring code documentation or internet forums to remind the most basic things. Writing custom code means handling things on fairly low level, even in *batteries included* language like Python. For a good reason. Using the `´requests` library expects user to have basic understanding about HTTP requests and responses, know how to access the response payload (should the request succeed), etc. The API does not and should not know anything about handling files on operating system level. That's a job for `os` package. You as the user have to know how to handle that.

So, would it not be nice to simply call that basic `wget` command you know by heart and use all the time? You can!

In [9]:
!wget -q -O /tmp/malware-pcap.zip.wget https://malware-traffic-analysis.net/2022/01/03/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap.zip

In [10]:
!ls -lah /tmp/malware-pcap*

-rw-r--r-- 1 jovyan users 1.3M Feb  2 10:21 /tmp/malware-pcap.zip
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.bashmagic
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.wget


Jupyter supports *magic* commands, either via built-in functions or calling shell commands. Exclamation mark as first symbol in the cell signifies a shell command. You can also use variables, though I imagine more complex logic will quickly become messy. Those magic commands are simply meant to be used for saving time on basics. Real power of a notebook still lies in all the options it gives the user for analyzing the data. And custom code enables that. Notebook users want to start analyzing data as fast as possible, so calling a familiar command instead of writing custom code can be a huge help.

In [11]:
OUTPUT_BASH_MAGIC = OUTPUT + ".bashmagic"

In [12]:
!wget -q -O $OUTPUT_BASH_MAGIC $URL

In [13]:
!ls -lah /tmp/malware-pcap*

-rw-r--r-- 1 jovyan users 1.3M Feb  2 10:21 /tmp/malware-pcap.zip
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.bashmagic
-rw-r--r-- 1 jovyan users 1.3M Jan  4  2022 /tmp/malware-pcap.zip.wget


#### Updating Suricata rules

Starting the code cell with percentage sign `%` invokes built-in functions. Again, it is a method for saving time on basic tasks. Jupyter has commands for loading python file content into a code cell, measuring execution time, installing python dependencies etc. Dependency install is especially useful in this notebook.

Working with Suricata eventually requires downloading and customizing rule files. Initial ruleset setup is quite easy, but maintaining it daily is a lot more difficult. Not every signature is meant to provide useful info in every environment. Alerting on a UNIX ping on a secure subnet that should only have Windows devices can be a red flag, but the same rule on a typical Linux server subnet is just a source of noise. A Linux system administrator would likely want to disable that rule, and ensure it remains disabled when ruleset is updated the next day. For a long time, this was out of scope for the main project and people had to resort to using legacy tools or custom scripts for downloading, updating, or modifying their rulesets.

[Suricata Update](https://suricata-update.readthedocs.io/en/latest/) is a command-line rule management tool developed and maintained by OISF. It comes bundled with Suricata, assuming it's built with all python tooling correctly enabled. But it can very easily installed individually, as just like Jupyter, it too is written in python. Jupyter builtin `%pip` command turns out to be very useful here, as we can ensure it's installed with minimal effort. Directly from the notebook. Makes it pretty handy for ensuring that notebook uses correct ruleset, without actually bundling the rules with notebook itself. File paths and software licenses can be a pain to deal with.

In [14]:
%pip install suricata-update

Note: you may need to restart the kernel to use updated packages.


Once suricata-update is installed, we can use it to enable rule sources, apply rule modification or disable overrides, and update our ruleset itself. For now, we simply enable a hunting ruleset that's likely to be too verbose on normal production installation. But it can highlight useful events that might go unnoticed with core Emerging Threats Open. For demonstration, the following cells:

* list available public sources to see what can be enabled;
* enable a new ruleset that's already defined in default public rule sources;
* call suricata-update itself to actually update the rule file;

Once called, `suricata-udpate` will download tarballs from each enabled source, apply conversion rules as needed, then concatenate the result into a single output rule file. If a rule file was downloaded recently, it might skipe the download entirely as normally this is done at most once a day.

In [15]:
!/home/jovyan/.local/bin/suricata-update list-sources

2/2/2023 -- 10:22:14 - <Info> -- Using data-directory /var/lib/suricata.
2/2/2023 -- 10:22:14 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
2/2/2023 -- 10:22:14 - <Info> -- Using /opt/suricata/share/suricata/rules for Suricata provided rules.
2/2/2023 -- 10:22:14 - <Info> -- Found Suricata version 7.0.0-beta1 at /opt/suricata/bin/suricata.
Name: et/open
  Vendor: Proofpoint
  Summary: Emerging Threats Open Ruleset
  License: MIT
Name: et/pro
  Vendor: Proofpoint
  Summary: Emerging Threats Pro Ruleset
  License: Commercial
  Replaces: et/open
  Parameters: secret-code
  Subscription: https://www.proofpoint.com/us/threat-insight/et-pro-ruleset
Name: oisf/trafficid
  Vendor: OISF
  Summary: Suricata Traffic ID ruleset
  License: MIT
Name: scwx/enhanced
  Vendor: Secureworks
  Summary: Secureworks suricata-enhanced ruleset
  License: Commercial
  Parameters: secret-code
  Subscription: https://www.secureworks.com/contact/ (Please reference CTU Countermeasures)
Name:

In [16]:
!/home/jovyan/.local/bin/suricata-update enable-source tgreen/hunting

2/2/2023 -- 10:22:14 - <Info> -- Using data-directory /var/lib/suricata.
2/2/2023 -- 10:22:14 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
2/2/2023 -- 10:22:14 - <Info> -- Using /opt/suricata/share/suricata/rules for Suricata provided rules.
2/2/2023 -- 10:22:14 - <Info> -- Found Suricata version 7.0.0-beta1 at /opt/suricata/bin/suricata.
2/2/2023 -- 10:22:14 - <Warning> -- The source tgreen/hunting is already enabled.
2/2/2023 -- 10:22:14 - <Info> -- Source tgreen/hunting enabled


In [17]:
!/home/jovyan/.local/bin/suricata-update

2/2/2023 -- 10:22:14 - <Info> -- Using data-directory /var/lib/suricata.
2/2/2023 -- 10:22:14 - <Info> -- Using Suricata configuration /etc/suricata/suricata.yaml
2/2/2023 -- 10:22:14 - <Info> -- Using /opt/suricata/share/suricata/rules for Suricata provided rules.
2/2/2023 -- 10:22:14 - <Info> -- Found Suricata version 7.0.0-beta1 at /opt/suricata/bin/suricata.
2/2/2023 -- 10:22:14 - <Info> -- Loading /etc/suricata/suricata.yaml
2/2/2023 -- 10:22:14 - <Info> -- Disabling rules for protocol pgsql
2/2/2023 -- 10:22:14 - <Info> -- Disabling rules for protocol modbus
2/2/2023 -- 10:22:14 - <Info> -- Disabling rules for protocol dnp3
2/2/2023 -- 10:22:14 - <Info> -- Disabling rules for protocol enip
2/2/2023 -- 10:22:14 - <Info> -- Last download less than 15 minutes ago. Not downloading https://rules.emergingthreats.net/open/suricata-7.0.0/emerging.rules.tar.gz.
2/2/2023 -- 10:22:14 - <Info> -- Last download less than 15 minutes ago. Not downloading https://raw.githubusercontent.com/travis

#### Parsing the PCAP with Suricata

Having set up our rule file and downloaded a PCAP to analyze, we can now proceed with parsing it with suricata. Most people know that Suricata can read PCAP files offline with `-r` flag. Not many are aware that Suricata logging directory can be overridden using `-l` flag and that Suricata can be pointed toward a rule file with `-S` or `-s` flags. Capital `-S` means exclusive rule file, meaning all rule files configured in `suricata.yaml` are ignored. Lowercase `-s` adds that file to list of files already in the main configuration. 

We want predictable output for the notebook, so we choose exclusive load. We also clean any existing logs from the logging directory, so we ensure that it's fully recreated. Suricata would append logs to any existing PCAP files, meaning rerunning the code cell would create duplicate events.

In [18]:
LOGDIR = "/tmp/logs"

In [19]:
!rm -rf $LOGDIR && mkdir $LOGDIR && ls -lah $LOGDIR

total 0
drwxr-xr-x 1 jovyan users   0 Feb  2 10:22 .
drwxrwxrwt 1 root   root  286 Feb  2 10:22 ..


In [20]:
LOG4J_PCAP = FILES[0]

In [21]:
!suricata -S /var/lib/suricata/rules/suricata.rules -l $LOGDIR -r $LOG4J_PCAP -v

2/2/2023 -- 10:22:18 - <Notice> - This is Suricata version 7.0.0-beta1 RELEASE running in USER mode
2/2/2023 -- 10:22:18 - <Info> - CPUs/cores online: 12
2/2/2023 -- 10:22:18 - <Info> - fast output device (regular) initialized: fast.log
2/2/2023 -- 10:22:18 - <Info> - eve-log output device (regular) initialized: eve.json
2/2/2023 -- 10:22:18 - <Info> - stats output device (regular) initialized: stats.log
2/2/2023 -- 10:22:22 - <Info> - 1 rule files processed. 33494 rules successfully loaded, 0 rules failed
2/2/2023 -- 10:22:22 - <Info> - Threshold config parsed: 0 rule(s) found
2/2/2023 -- 10:22:22 - <Info> - 33497 signatures processed. 1329 are IP-only rules, 5290 are inspecting packet payload, 26674 inspect application layer, 108 are decoder event only
2/2/2023 -- 10:22:28 - <Notice> - Threads created -> RX: 1 W: 12 FM: 1 FR: 1   Engine started.
2/2/2023 -- 10:22:28 - <Info> - Starting file run for /tmp/2022-01-01-thru-03-server-activity-with-log4j-attempts.pcap
2/2/2023 -- 10:22:28 

This should result in some interesting data to analyze in `/tmp/logs/eve.json`. Before we introduce data scientists tooling, let's just extract a single high severity alert to see how a typical EVE event looks like. As you can see, it's a highly nested JSON event with a lot of extra context. Very useful for providing analyst with as much data as possible. But it can be quite daunting to explore, as a single event can already fill entire screen but a production network can produce millions.

The example scans the log file until first `alert` is found. Alert is a good first event to look at, since it also includes info from other event types.

In [22]:
import json

In [23]:
with open("/tmp/logs/eve.json", "r") as handle:
    handle.readline()
    for line in handle:
        eve = json.loads(line)
        if eve.get("event_type", "") == "alert" and eve.get("alert", {}).get("severity") == 1:
            print(json.dumps(eve, indent=2))
            break

{
  "timestamp": "2022-01-01T19:05:52.130715+0000",
  "flow_id": 2151509425429418,
  "pcap_cnt": 10350,
  "event_type": "alert",
  "src_ip": "186.220.97.233",
  "src_port": 4873,
  "dest_ip": "198.71.247.91",
  "dest_port": 80,
  "proto": "TCP",
  "pkt_src": "wire/pcap",
  "tx_id": 0,
  "alert": {
    "action": "allowed",
    "gid": 1,
    "signature_id": 2029022,
    "rev": 3,
    "signature": "ET SCAN Mirai Variant User-Agent (Inbound)",
    "category": "Attempted Administrator Privilege Gain",
    "severity": 1,
    "metadata": {
      "affected_product": [
        "Linux"
      ],
      "attack_target": [
        "IoT"
      ],
      "created_at": [
        "2019_11_21"
      ],
      "deployment": [
        "Perimeter"
      ],
      "former_category": [
        "SCAN"
      ],
      "signature_severity": [
        "Minor"
      ],
      "updated_at": [
        "2020_10_29"
      ]
    }
  },
  "http": {
    "hostname": "127.0.0.1",
    "http_port": 80,
    "url": "/shell?cd+/tmp;

### Dataframes and Pandas

JSON is great for building applications and for security analytics. It is structured and most modern no-sql databases default to using it. Nowadays, security analysts are used to reading it. Data scientists and data engineers, however, working with tabular row-column data. Most data mining and machine learning algorithms work on data vectors or matrices, with a matrix essentially just being a vector of vectors. For statistical analysis, those vectors usually contain floating point numbers - some kind of numeric measurements. Each vector would make up a column of data within the matrix, and complex calculations are carried out on them. 

Sometimes multiple columns are combined to transform raw data into more meaningful context. For example, Suricata measures request and response bytes separately, and analyst might want to sum up those columns. Other times a vector would be scaled or normalized, as measurements might be on different scales and would thus not be directly comparable. For example, defining a generic threshold is very difficult as traffic scale and patterns differ greatly between organizations.

Column of data might not be numeric measurements, but rather textual values, booleans, timestamps, categories etc. This is usually the case with NSM data. A classical matrix stores only one *type* of data, usually numbers. A *dataframe* is basically a matrix where each column can be of different type. [Pandas](https://pandas.pydata.org/) is a data analysis and manipulation library that brings dataframes to Python language.

Getting started with dataframes is quite simple. The challenge is changing the mindset. Coders experienced in imperative languages might need to relearn what they already know. Statistical analysis is centered around vectors, rather than individual elements. Furthermore, API-s are often declarative and follow functional programming paradigms. Basically, you need to drop the *for loop* and learn how to *apply* functions instead.

Reason is performance. CPU-s are really efficient at crunching numbers and SIMD instructions speed up calculations by orders of magnitude. Even the smartest code has a hard time competing with that. Data science libraries in high-level languages such as Python often function as interfaces. They give the user an extensive API that's intuitive to use, yet actually defer the calculations to low level code written in C or C++ that are able to leverage optimized CPU instructions or even GPUs. In other words, passing a vector of data means it might be handled by efficient machine code, whereas looping over items in Python means they will be always evaluated by Python. A language notorious for being slow.

In the case of Suricata data, we're mostly working with textual values that would firstly need to be converted into numerical measurements before such gains could be made. Hence, we likely won't see a huge performance increase doing things the *pandas way* rather than *python way*. In fact, sometimes it will be faster to just convert data between python objects and native python data structures, especially when fast random lookup is required (lists are really bad at that). This is a challenge when combining NSM data with data science tools. Concessions need to be made on both sides. Data science tools are not better nor worse than tech stacks that are well established in security. They simply open new possibilities.

Benefits of choosing pandas over native python are:

* Pandas implements a ton of useful methods for working with and filtering data;
* Pandas code can be much more concise that imperative logic in python, often achieving in few lines that would otherwise require complex functions;
* Jupyter has native display and visualization support for pandas dataframes, making visual exploration much easier;

Choose raw python objects over dataframes when:
* complex conversion needs to be made that is not readily provided in Pandas;
* fast random access is needed;

In [24]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd

In [26]:
pd.options.display.html.use_mathjax = False

In [27]:
pd.DataFrame([{"src_ip": "1.1.1.1", "flow_id": 123}, {"src_ip": "2.2.2.2", "flow_id": 124}])

,src_ip,flow_id
0,1.1.1.1,123
1,2.2.2.2,124


#### Nested JSON and Pandas

As anyone familiar with Suricata EVE format knows, it can be challenging to work with thanks to nested structure and sheer amount of fields. Suricata can log well over 1000 distinct JSON key-value pairs, omitting any that has been disabled in configuration or simply cannot be parsed from a particular flow. For example, a TLS 1.3 connection will for now most likely display a SNI (Server Name Indication) but will not have any certificate fields, as latter are simply not visible in plaintext network view. Sometimes a field only appears in handful of events, making it easy to overlook. Figuring out what is available to work on is a challenge.

Pandas provides a useful method `json_normalize` for normalizing nested JSON fields into dataframe. Resulting columns use dot notation to signify nested object, similar to how Elasticsearch does it. For example, `sni` key is part of `tls` section and would be accessible from column `tls.sni`. Missing values are noted as `np.nan`, or *not a number*, which is a statistical analysis convention. As mentioned, statistics is where pandas and underlying *numpy* libraries originated from. A measurement could simply be missing due to bad instrumentation, or it might be result of some algorithm that does not provide meaningful output is some scenarios. For example, dividing by zero is not allowed but nevertheless happens very easily in statistics. When trying to find a ratio between two measurements and second counter is 0, the only possible result is `NaN` as result `0` would be mathematically incorrect.

*Not a number* is actually a special floating point value and a perfectly legal data type for vector computing. For Suricata, however, it simply means a missing value and likely has nothing to do with numeric measurements. Think of it as `null` or `None` value.

Note that we still need some regular python code to parse individual EVE messages, as built-in pandas `read_json` would assume a full JSON structure rather than *newline delimited* JSON (NDJSON) log. Also note that this method reads all logs into memory and all further processing is also done there as well. Do not expect it to scale over gigabytes of data, unless of course you have access to a lot of RAM on single machine. It's meant for limited data exploration, quick prototyping, etc. Big data analytics when unable to fit into local memory requires supporting infrastructure.

In [28]:
with open("/tmp/logs/eve.json", "r") as handle:
    DF = pd.json_normalize([
        json.loads(line) for line in handle
    ])
DF

,timestamp,flow_id,event_type,src_ip,src_port,dest_ip,dest_port,proto,flow.pkts_toserver,flow.pkts_toclient,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
0,2022-01-01T00:00:13.076985+0000,1.777836e+15,flow,178.175.173.166,43719.0,198.71.247.91,23.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:01:49.092097+0000,1.521456e+15,dns,209.141.58.15,35550.0,198.71.247.91,53.0,UDP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:13.076985+0000,6.780576e+14,flow,54.83.160.152,NaN,198.71.247.91,NaN,ICMP,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:05:51.487413+0000,2.093425e+15,sip,193.46.255.60,5103.0,198.71.247.91,5060.0,UDP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:00:13.076985+0000,2.068446e+15,flow,18.207.93.95,NaN,198.71.247.91,NaN,ICMP,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25885,2022-01-01T00:00:13.076985+0000,2.637278e+13,flow,89.248.163.166,54222.0,198.71.247.91,5554.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25886,2022-01-01T00:00:13.076985+0000,2.599530e+14,flow,45.146.166.123,52482.0,198.71.247.91,35829.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25887,2022-01-01T00:00:13.076985+0000,2.198347e+15,flow,89.248.165.248,55041.0,198.71.247.91,8443.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25888,2022-01-01T00:00:13.076985+0000,9.765911e+14,flow,89.248.163.155,8080.0,198.71.247.91,1206.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Quick overview of columns

Before proceeding analyzing the data, we need a initial overview of what we can work with. The most simple measurement is simply understanding the number of rows and columns in the set.

In [29]:
DF.shape

(25890, 547)

In [30]:
print("dataframe has %d rows and %d columns" % DF.shape)

dataframe has 25890 rows and 547 columns


As mentioned before, understanding the available fields is particularly important for EVE data. This info can be accessed directly from the dataframe object. The reader might also observe significant noise in these values, as the simple EVE log we loaded most likely contains `stats` events. These fields provide a lot of statistics from Suricata engine and are really useful for finding performance problems. However, they don't contribute much to threat hunting and simply overshadow data fields.

To deal with this, we can write simple python to filter them out and provide a much cleaner view of the data.

In [31]:
COLS_STATS = [c for c in list(DF.columns.values) if c.startswith("stats")]
len(COLS_STATS)

431

In [32]:
print("%d stats cols from total %d" % (len(COLS_STATS), len(DF.columns.values)))

431 stats cols from total 547


In [33]:
COLS_DATA = [c for c in list(DF.columns.values) if not c.startswith("stats")]

In [34]:
print("%d data columns" % len(COLS_DATA))

116 data columns


In [35]:
DF[COLS_DATA]

,timestamp,flow_id,event_type,src_ip,src_port,dest_ip,dest_port,proto,flow.pkts_toserver,flow.pkts_toclient,...,alert.metadata.signature_severity,quic.version,files,app_proto_ts,metadata.flowints.http.anomaly.count,snmp.usm,metadata.flowints.tcp.retransmission.count,tcp.ecn,tcp.cwr,metadata.flowbits
0,2022-01-01T00:00:13.076985+0000,1.777836e+15,flow,178.175.173.166,43719.0,198.71.247.91,23.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01T00:01:49.092097+0000,1.521456e+15,dns,209.141.58.15,35550.0,198.71.247.91,53.0,UDP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01T00:00:13.076985+0000,6.780576e+14,flow,54.83.160.152,NaN,198.71.247.91,NaN,ICMP,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01T00:05:51.487413+0000,2.093425e+15,sip,193.46.255.60,5103.0,198.71.247.91,5060.0,UDP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01T00:00:13.076985+0000,2.068446e+15,flow,18.207.93.95,NaN,198.71.247.91,NaN,ICMP,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25885,2022-01-01T00:00:13.076985+0000,2.637278e+13,flow,89.248.163.166,54222.0,198.71.247.91,5554.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25886,2022-01-01T00:00:13.076985+0000,2.599530e+14,flow,45.146.166.123,52482.0,198.71.247.91,35829.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25887,2022-01-01T00:00:13.076985+0000,2.198347e+15,flow,89.248.165.248,55041.0,198.71.247.91,8443.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25888,2022-01-01T00:00:13.076985+0000,9.765911e+14,flow,89.248.163.155,8080.0,198.71.247.91,1206.0,TCP,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Describe method

A `describe` method is a useful shortcut for understanding statistical properties of numeric columns. It has limited value for NSM data, as most fields are either textual or categorical. Some numerical EVE values can not be analyzed like this, for example source and destination ports, randomly generated flow ID values, etc. However, it can provide great insights into `flow` and `stats` records, instantly revealing data properties such as distribution, max and min values, mean values, standard deviation, etc.

In [36]:
DF.describe()

,flow_id,src_port,dest_port,flow.pkts_toserver,flow.pkts_toclient,flow.bytes_toserver,flow.bytes_toclient,flow.age,pcap_cnt,dns.id,...,stats.app_layer.error.nfs_udp.internal,stats.app_layer.error.krb5_udp.alloc,stats.app_layer.error.krb5_udp.parser,stats.app_layer.error.krb5_udp.internal,stats.app_layer.expectations,stats.http.memuse,stats.http.memcap,stats.ftp.memuse,stats.ftp.memcap,stats.file_store.open_files
count,2.588900e+04,23525.000000,23525.000000,23242.000000,23242.000000,23242.000000,23242.000000,23093.000000,2751.000000,59.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.137046e+15,42137.309628,14506.841445,1.315722,0.392565,111.311247,51.871870,4.375828,21690.181752,25804.694915,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,6.468296e+14,17980.868433,17827.274187,1.842366,1.576366,710.506763,409.702884,37.485287,12308.976539,22754.924353,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,5.713087e+08,0.000000,0.000000,1.000000,0.000000,42.000000,0.000000,0.000000,18.000000,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,5.888345e+14,36103.000000,1694.000000,1.000000,0.000000,54.000000,0.000000,0.000000,10653.000000,5463.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.130907e+15,48856.000000,7000.000000,1.000000,0.000000,54.000000,0.000000,0.000000,23270.000000,16765.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.695347e+15,54814.000000,22037.000000,1.000000,0.000000,58.000000,0.000000,0.000000,33568.000000,43119.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2.251782e+15,65531.000000,65528.000000,134.000000,91.000000,66168.000000,25717.000000,899.000000,39193.000000,64206.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
